from https://github.com/artivis/manif/blob/devel/examples/se2_localization.py

In [1]:
from typing import List
from manifpy import SE2, SE2Tangent

import numpy as np
from numpy.linalg import inv

assert SE2.Dim == 2 # 2D

In [2]:
Vector = np.array

def Covariance():
    return np.zeros((SE2.DoF, SE2.DoF))

def Jacobian():
    return np.zeros((SE2.DoF, SE2.DoF))

## START CONFIGURATION

In [3]:
NUMBER_OF_LMKS_TO_MEASURE = 3

In [4]:
U_NOM = Vector([0.1, 0.0, 0.05])
U_SIGMAS = Vector([0.1, 0.1, 0.1])
U = np.diagflat(np.square(U_SIGMAS))
#print('U', U)

In [5]:
# Define five landmarks in R^2
LANDMARKS = [
    Vector([2.0,  0.0]),
    Vector([2.0,  1.0]),
    Vector([2.0, -1.0])
]

Y_SIGMAS = Vector([0.01, 0.01])
R = np.diagflat(np.square(Y_SIGMAS))
#print('R', R)

In [6]:
def make_measurements(x_simulation: SE2, landmarks: List):
    # Define the beacon's measurements
    measurements = []

    # then we measure all landmarks
    for i in range(NUMBER_OF_LMKS_TO_MEASURE):
        b = landmarks[i]                                # lmk coordinates in world frame

        # simulate noise
        y_noise = Y_SIGMAS * np.random.rand(SE2.Dim)    # measurement noise
        assert y_noise.shape == (2,)
        
        y = x_simulation.inverse().act(b)               # landmark measurement, before adding noise

        y = y + y_noise                                 # landmark measurement, noisy
        measurements.append(y)                          # store for the estimator just below    
        
    return measurements

In [9]:
# Define the robot pose element and its covariance
X_simulation = SE2.Identity()
X = SE2.Identity()
X_unfiltered = SE2.Identity()

P = Covariance()

# Declare the Jacobians of the motion wrt robot and control
J_x = Jacobian()
J_u = Jacobian()

# Declare some temporaries
J_xi_x = Jacobian()
J_e_xi = np.zeros((SE2.Dim, SE2.DoF))


# pretty print
np.set_printoptions(precision=3, suppress=True)

print('X STATE     :    X      Y      Z    TH_x   TH_y   TH_z ')
print('-------------------------------------------------------')
print('X initial   : ', X_simulation.log().coeffs())
print('-------------------------------------------------------')

assert isinstance(X_simulation.log(), SE2Tangent)
assert isinstance(X_simulation.log().coeffs(), np.ndarray)


# Make 10 steps. Measure up to three landmarks each time.
for t in range(10):

    # simulate noise
    u_noisy = U_NOM + U_SIGMAS * np.random.rand(SE2.DoF) # noisy control
    u_simu = SE2Tangent(U_NOM)
    u_est = SE2Tangent(u_noisy)
    u_unfilt = SE2Tangent(u_noisy)

    
    # I. Simulation
    X_simulation = X_simulation + u_simu                # overloaded X.rplus(u) = X * exp(u)
    assert isinstance(X_simulation, SE2)   

    # the beacon's measurements
    measurements = make_measurements(X_simulation, LANDMARKS)

        
    # II. Estimation
    # First we move
    X = X.plus(u_est, J_x, J_u)                         # X * exp(u), with Jacobians
    P = J_x * P * J_x.transpose() + J_u * U * J_u.transpose()

    # Then we correct using the measurements of each lmk
    for i in range(NUMBER_OF_LMKS_TO_MEASURE):
        # landmark
        b = LANDMARKS[i]                                # lmk coordinates in world frame

        # measurement
        y = measurements[i]                             # lmk measurement, noisy

        # expectation
        e = X.inverse(J_xi_x).act(b, J_e_xi)            # note: e = R.tr * ( b - t ), for X = (R,t).
        H = J_e_xi @ J_xi_x                             # Jacobian of the measurements wrt the robot pose. 
                                                        # note: H = J_e_x = J_e_xi * J_xi_x
        E = H @ P @ H.transpose()

        # innovation
        z = y - e
        Z = E + R

        # Kalman gain
        K = P @ H.transpose() @ inv(Z)                  # K = P * H.tr * ( H * P * H.tr + R).inv

        # Correction step
        dx = K @ z                                      # dx is in the tangent space at X

        # Update
        X = X + SE2Tangent(dx)                          # overloaded X.rplus(dx) = X * exp(dx)
        P = P - K @ Z @ K.transpose()

        
    # III. Unfiltered
    # move also an unfiltered version for comparison purposes
    X_unfiltered = X_unfiltered + u_unfilt

    
    # IV. Results
    print('X simulated : ', X_simulation.log().coeffs().transpose())
    print('X estimated : ', X.log().coeffs().transpose())
    print('X unfilterd : ', X_unfiltered.log().coeffs().transpose())
    print('-------------------------------------------------------')

X STATE     :    X      Y      Z    TH_x   TH_y   TH_z 
-------------------------------------------------------
X initial   :  [0. 0. 0.]
-------------------------------------------------------
X simulated :  [0.1  0.   0.05]
X estimated :  [ 0.095 -0.003  0.049]
X unfilterd :  [0.14  0.067 0.1  ]
-------------------------------------------------------
X simulated :  [ 0.2 -0.   0.1]
X estimated :  [ 0.196 -0.005  0.101]
X unfilterd :  [0.255 0.1   0.171]
-------------------------------------------------------
X simulated :  [ 0.3  -0.    0.15]
X estimated :  [ 0.296 -0.005  0.15 ]
X unfilterd :  [0.412 0.206 0.225]
-------------------------------------------------------
X simulated :  [ 0.4 -0.   0.2]
X estimated :  [ 0.398 -0.004  0.2  ]
X unfilterd :  [0.603 0.3   0.338]
-------------------------------------------------------
X simulated :  [0.5  0.   0.25]
X estimated :  [ 0.496 -0.006  0.249]
X unfilterd :  [0.723 0.353 0.468]
------------------------------------------------------